In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, Conv1D, MaxPooling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow import keras, config
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("GPUs Available: ", len(config.list_physical_devices('GPU')))

In [ ]:
%load_ext autoreload
%autoreload 2
from image_process.image_preprocessors import add_noise, add_noise_randomly, shift_random_update_positions, shift_randomly_position_labels
from data_filters import tempo_interval, take_percent
# Global settings
TEST_SPLIT_SIZE = 0.80
VALIDATION_SPLIT_SIZE = 0.90
IMAGE_TARGET_SIZE = (5, 1400, 1)
BATCH_SIZE = 128
EPOCH_SIZE = 500
LETTER_END_POSITION = "P1"
ADD_NOISE_RANDOMLY = [-2, 20]
ADD_SIGNAL_INDENT_RANDOMLY = 12860
SHIFT_MIN_MAX = [-200, -140]
TEMPO_INTERVAL = [16, 22]
IMAGE_PREPOCESSORS = [
    {"func": shift_randomly_position_labels, "params" : SHIFT_MIN_MAX},
]

In [ ]:
def show_image(img, width, position):
    plt.figure(figsize=(30,5))
    plt.xlim(0, width)
    plt.xticks(position)    
    plt.imshow(img)
    plt.show()

def normalize_output(n:int):
    return position_labels_de_normalizer(n, max_position=set_obj.max_first_letter_position, min_position=set_obj.min_first_letter_position, shift_max=shift_max)

In [ ]:
from Image_Generator_helpers import  DataSets, set_paths, global_path,Random_Item
from data_filters import tempo_interval_raw, position_labels_de_normalizer
set_obj: DataSets = DataSets(set_paths, global_path, [tempo_interval_raw(TEMPO_INTERVAL)], ["P1", "P2", "P3", "P4"])
set_obj.csv_files

shift_max = abs(SHIFT_MIN_MAX[0] - SHIFT_MIN_MAX[1])

def get_position_labels(random_items):
    return [item.csv_row[LETTER_END_POSITION].values.astype(np.float64)[0] for item in random_items]

def position_labels_normalizer(labels, set_obj: DataSets, random_items: list[Random_Item]): # reuse other de_normalizer function
    return np.array([(label - set_obj.min_first_letter_position) / (set_obj.max_first_letter_position + shift_max - set_obj.min_first_letter_position) for label in labels]) # Store these parameters on json

def letter_e_bias(dataFrame):
    return dataFrame["WORD"].apply(str).str.startswith("e")



In [ ]:
from keras_generators.image_generator import Image_Generator_RAW
class Regression_Image_Generator:
    def __init__(self, image_amount, image_preprocessors, noise_range):
        self.generator = Image_Generator_RAW(
            image_amount=image_amount,
            set_obj= set_obj,
            FFT_JUMP=64,
            batch_size=BATCH_SIZE,
            image_target_size=IMAGE_TARGET_SIZE,
            image_prepocessors=image_preprocessors,
            noise_range=noise_range,
            random_signal_indent=ADD_SIGNAL_INDENT_RANDOMLY,
            label_func = get_position_labels,
            label_post_process=position_labels_normalizer,
            signal_cut_off=12860,
            position_probability=[0,2]
        )

Train_Generator_Raw = Regression_Image_Generator(
    image_amount=BATCH_SIZE * EPOCH_SIZE,
    image_preprocessors=IMAGE_PREPOCESSORS,
    noise_range=ADD_NOISE_RANDOMLY
).generator


In [ ]:
# t, l = Train_Generator_Raw.__getitem__(0)

# labels_de_normalized = normalize_output(l)

# for idx,img in enumerate(t):
#     print("label: ")
#     pos = labels_de_normalized[idx]
#     print(pos)
#     print(img.shape)
#     show_image(img, 200, [50, 200, pos])

In [ ]:
def conv_model_position(input_layer):
    x = keras.layers.Cropping2D(cropping=((0, 0), (0,200)), data_format=None)(input_layer)

    x = Conv2D(90,(1,7), padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(2,1),padding="same")(x)

    x = Conv2D(90,(1,7),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(1,5),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Flatten()(x)

    return x

In [ ]:
from tensorflow.keras import layers

input_layer = Input(shape=(5, 400, 1))
conv_model_position_flattened = conv_model_position(input_layer)
output_layer_position = Dense(1, name="regr")(conv_model_position_flattened)

model = Model(inputs=input_layer, outputs=output_layer_position)
model.compile(loss=["mse"], optimizer='adam', metrics=["mean_absolute_error"])

print(model.summary())


In [ ]:
init_epoch = 0

In [ ]:

num_epochs = 5

def fit_model(epochs):
	
	global init_epoch
	history = model.fit(
		Train_Generator_Raw,
		steps_per_epoch = EPOCH_SIZE,
		epochs = epochs + init_epoch,
		initial_epoch=init_epoch,
		verbose =1,
		# validation_data = validation_batch_generator,
		# validation_steps = int(len(train_validation) // BATCH_SIZE),
		workers=12,
		use_multiprocessing=True
	)

	
	init_epoch += epochs
	return history

history = fit_model(num_epochs)


In [ ]:
%%capture cap --no-stderr
from log_helpers.training_log import Training_Data_Log, print_name, json_to_file, get_deviating_predictions
import inspect

data_log = Training_Data_Log()
data_log.model_config = model.to_json()
data_log.model_config_method_string = [inspect.getsource(conv_model_position)]
# data_log.training_sets = get_sets()
# data_log.training_set_size = len(train)
# data_log.validation_set_size = len(train_validation)
# data_log.test_set_size = len(train_test)
data_log.image_pre_processors = print_name(IMAGE_PREPOCESSORS)
data_log.noise_added = ADD_NOISE_RANDOMLY
# data_log.training_data_masks = print_name(MASKS)
data_log.model_summary = model.summary()
data_log.model_optimizer = str(type(model.optimizer))
data_log.model_history = history.history
data_log.model_history_final_epoch = {k: v[-1] for k, v in history.history.items()}
data_log.total_epochs = init_epoch


In [ ]:
class Regression_Results:
    image_preprocessors_test = None
    total_predictions = None
    noise_level = None
    difference_in_pixels = None
    predictions_off_by_more_than_difference = None
    predictions_incorrect_prercent = None
    model_evaluation = None

noise_levels = [[-2, 20], [0, 20]]
result_array = []
BATCHES_TESTING = 250
pixel_difference = 3

def regression_comparer(prediction, correct):
    prediction = normalize_output(prediction)
    correct = normalize_output(correct)
    return abs(prediction[0] - correct) < pixel_difference

for noise_level in noise_levels:

    test_batch_generator = Regression_Image_Generator(
        image_amount=BATCH_SIZE * BATCHES_TESTING,
        image_preprocessors=IMAGE_PREPOCESSORS,
        noise_range=noise_level
    ).generator

    regression_differences  = get_deviating_predictions(test_batch_generator, model, regression_comparer)

    evaluations = model.evaluate(test_batch_generator, verbose = 0)

    results = Regression_Results()
    results.total_predictions = BATCHES_TESTING * BATCH_SIZE
    results.noise_level = noise_level
    results.difference_in_pixels = pixel_difference
    results.predictions_off_by_more_than_difference = len(regression_differences)
    results.predictions_incorrect_prercent =len(regression_differences) / (BATCHES_TESTING * BATCH_SIZE) * 100
    results.model_evaluation = evaluations

    result_array.append(results.__dict__)

    print(len(regression_differences))


In [ ]:
data_log.results = result_array
print(data_log.results)

In [ ]:
json_to_file("logs/regression/regression_data_log_raw", data_log)

In [ ]:

for idx, diff in enumerate(regression_differences):

    if idx > 100:
        break

    pred, correct, img = diff

    pred = normalize_output(pred)
    correct = normalize_output(correct)

    img_pred = img.copy()
    img_correct = img.copy()

    print('Prediction', round(pred[0]))
    img_pred[:, round(int(pred))] = 1
    show_image(img_pred, 300, [round(pred[0])])

    print('Correct', round(correct))
    img_correct[:, round(int(correct))] = 1
    show_image(img_correct, 300, [round(pred[0])])

    print("----------------------------------------------------------------------------------------")
